## 파인콘 백터 DB 저장
A × ( B / C )

- **A**: 전세사기피해주택의 임대인이 체납한 고지 또는 신고 건별 국세 금액  
- **B**: 전세사기피해주택의 가격  
- **C**: 전세사기피해주택의 임대인이 보유한 모든 주택의 가격 합계액

In [10]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain.vectorstores import Pinecone
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

load_dotenv()

True

In [11]:
loader = Docx2txtLoader('law_2.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

In [12]:
embedding = OpenAIEmbeddings(model = 'text-embedding-3-large')

api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)

database = PineconeVectorStore.from_documents(
    embedding=embedding,
    index_name = 'law-2',
    documents= document_list,
)


TypeError: Pinecone.__init__() got an unexpected keyword argument 'api_key'

In [ ]:
query = '전세사기 피해자 금융지원에 대해 설명해주세요.'

retrieved_docs = database.similarity_search(query=query)

prompt = '''
[identity]
-당신은 전세사기 피해 법률 전문가입니다.
-[context]를 참고하여 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question: {query}
'''

In [ ]:
formatted = prompt.format(retrieved_docs=retrieved_docs, query=query)

llm = ChatOpenAI(model = 'gpt-4o')

ai_message = llm.invoke(formatted)
ai_message

AIMessage(content='전세사기피해자를 위한 금융지원은 주거안정과 신속한 회복을 목적으로 다양한 지원책으로 구성되어 있습니다. 전세사기 피해자 법령에 따르면, 금융지원의 주요 내용으로는 다음과 같은 것들이 포함될 수 있습니다:\n\n1. **금융지원 대상**: 전세사기 피해자로 인정되는 임차인은 금융 지원을 받을 자격이 있습니다. 이들은 법적 절차를 거쳐 전세사기 피해자로 확정된 후 해당 지원을 신청할 수 있습니다.\n\n2. **주택 매입 및 임대료 지원**: 전세사기 피해주택을 국가 또는 지방자치단체가 매입하여 피해자들에게 우선적으로 임대하거나, 임대료의 일정 부분을 감면하여 주거 안정을 도모합니다.\n\n3. **재정 지원을 통한 임대료 감면**: 공공임대주택에 거주하는 경우, 감면된 임대료를 재정으로 지원받을 수 있으며 이는 피해자의 부담을 줄이는 데 기여합니다.\n\n4. **우선 공급 및 재정 지원**: 공공주택사업자나 관련 기관이 피해주택을 매입하여 우선 공급할 경우, 그에 따른 재정 지원을 국가 또는 지방자치단체로부터 받을 수 있습니다.\n\n5. **법률 및 행정 지원**: 피해자들의 권리 회복과 관련된 법률 상담 및 행정 절차 지원도 중요한 부분입니다.\n\n이와 같은 금융 지원은 전세사기 피해자가 빠르게 안정된 생활로 복귀할 수 있도록 돕는 것을 목표로 합니다. 구체적인 신청 절차나 지원 범위 등은 국토교통부 및 관련 기관을 통해 확인하는 것이 좋습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 380, 'prompt_tokens': 3905, 'total_tokens': 4285, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction